In [ ]:
%matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 20: Grove Positioning System ---</h2><p>It's finally time to meet back up with the Elves. When you try to contact them, however, you get no reply. Perhaps you're out of range?</p>
<p>You know they're headed to the grove where the <em class="star">star</em> fruit grows, so if you can figure out where that is, you should be able to meet back up with them.</p>
<p>Fortunately, your handheld device has a file (your puzzle input) that contains the grove's coordinates! Unfortunately, the file is <em>encrypted</em> - just in case the device were to fall into the wrong hands.</p>
<p>Maybe you can <span title="You once again make a mental note to remind the Elves later not to invent their own cryptographic functions.">decrypt</span> it?</p>
<p>When you were still back at the camp, you overheard some Elves talking about coordinate file encryption. The main operation involved in decrypting the file is called <em>mixing</em>.</p>
<p>The encrypted file is a list of numbers. To <em>mix</em> the file, move each number forward or backward in the file a number of positions equal to the value of the number being moved. The list is <em>circular</em>, so moving a number off one end of the list wraps back around to the other end as if the ends were connected.</p>
<p>For example, to move the <code>1</code> in a sequence like <code>4, 5, 6, <em>1</em>, 7, 8, 9</code>, the <code>1</code> moves one position forward: <code>4, 5, 6, 7, <em>1</em>, 8, 9</code>. To move the <code>-2</code> in a sequence like <code>4, <em>-2</em>, 5, 6, 7, 8, 9</code>, the <code>-2</code> moves two positions backward, wrapping around: <code>4, 5, 6, 7, 8, <em>-2</em>, 9</code>.</p>
<p>The numbers should be moved <em>in the order they originally appear</em> in the encrypted file. Numbers moving around during the mixing process do not change the order in which the numbers are moved.</p>
<p>Consider this encrypted file:</p>
<pre><code>1
2
-3
3
-2
0
4
</code></pre>
<p>Mixing this file proceeds as follows:</p>
<pre><code>Initial arrangement:
1, 2, -3, 3, -2, 0, 4

1 moves between 2 and -3:
2, 1, -3, 3, -2, 0, 4

2 moves between -3 and 3:
1, -3, 2, 3, -2, 0, 4

-3 moves between -2 and 0:
1, 2, 3, -2, -3, 0, 4

3 moves between 0 and 4:
1, 2, -2, -3, 0, 3, 4

-2 moves between 4 and 1:
1, 2, -3, 0, 3, 4, -2

0 does not move:
1, 2, -3, 0, 3, 4, -2

4 moves between -3 and 0:
1, 2, -3, 4, 0, 3, -2
</code></pre>

<p>Then, the grove coordinates can be found by looking at the 1000th, 2000th, and 3000th numbers after the value <code>0</code>, wrapping around the list as necessary. In the above example, the 1000th number after <code>0</code> is <code><em>4</em></code>, the 2000th is <code><em>-3</em></code>, and the 3000th is <code><em>2</em></code>; adding these together produces <code><em>3</em></code>.</p>
<p>Mix your encrypted file exactly once. <em>What is the sum of the three numbers<msreadoutspan class="msreadout-line-highlight msreadout-inactive-highlight"> that form the grove <msreadoutspan class="msreadout-word-highlight">coordinates</msreadoutspan>?</msreadoutspan></em></p>
</article>


In [42]:
from collections import deque
from pprint import pprint

from more_itertools import one


tests = [
    {
        "name": "Example 1",
        "s": """
            1
            2
            -3
            3
            -2
            0
            4
        """,
        "expected": 3,
    },
]


@dataclass(repr=False)
class Node:
    value: int
    next: Node | None = None
    prev: Node | None = None

    def __repr__(self) -> str:
        return (
            f"Node(value={self.value}"
            f", next={self.next.value if self.next else None}"
            f", prev={self.prev.value if self.prev else None})"
        )


class Decryptor(Print):
    def __init__(self, s: str) -> None:
        self.nodes = self._create_list(s)
        self.head = self.nodes[0]
        self.zero = one(n for n in self.nodes if n.value == 0)

        self._mix()

    @classmethod
    def _create_list(cls, s: str) -> list[Node]:
        nodes = [Node(int(i.strip())) for i in s.strip().splitlines()]

        for i in range(1, len(nodes)):
            nodes[i - 1].next = nodes[i]
            nodes[i].prev = nodes[i - 1]

        nodes[0].prev = nodes[-1]
        nodes[-1].next = nodes[0]
        return nodes

    def _mix(self) -> None:
        for node in self.nodes:
            if node.value == 0:
                continue

            prv = node.prev
            nxt = node.next

            # reomove node
            prv.next = nxt
            nxt.prev = prv

            if self.head is node:
                self.head = nxt

            value = node.value
            current = node

            # move rigth if positve via next
            if value > 0:
                value %= len(self.nodes) - 1
                while value > 0:
                    current = current.next
                    value -= 1

            # move left if negative vie prev
            else:
                value = -value % (len(self.nodes) - 1)
                while value >= 0:
                    current = current.prev
                    value -= 1

            nxt = current.next
            current.next = node
            nxt.prev = node
            node.next = nxt
            node.prev = current

    def grove_sum(self, nths=(1_000, 2_000, 3_000)) -> int:
        collect = sorted((n % len(self.nodes) for n in nths), reverse=True)

        max_steps, grove_sum, step = collect[0], 0, 0
        current = self.zero
        next_collect = collect.pop()

        while step <= max_steps:
            if step == next_collect:
                grove_sum += current.value

                if collect:
                    next_collect = collect.pop()

            current = current.next
            step += 1

        return grove_sum

    def linkedlist_to_list(self) -> list[int]:
        current = self.head.next
        result = [self.head.value]

        while current is not self.head:
            result.append(current.value)
            current = current.next

        return result


@test(tests=tests)
def test_part_I(s: str) -> int:
    return Decryptor(s).grove_sum()


Test Example 1 passed, for test_part_I.
Success


In [43]:
with open("../input/day20.txt") as f:
    puzzle = f.read()

print(f"Part I: {Decryptor(puzzle).grove_sum():,}")

Part I: 8,302


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>8302</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The grove coordinate values seem nonsensical. While you ponder the mysteries of Elf encryption, you suddenly remember the rest of the decryption routine you overheard back at camp.</p>
<p>First, you need to apply the <em>decryption key</em>, <code>811589153</code>. Multiply each number by the decryption key before you begin; this will produce the actual list of numbers to mix.</p>
<p>Second, you need to mix the list of numbers <em>ten times</em>. The order in which the numbers are mixed does not change during mixing; the numbers are still moved in the order they appeared in the original, pre-mixed list. (So, if -3 appears fourth in the original list of numbers to mix, -3 will be the fourth number to move during each round of mixing.)</p>
<p>Using the same example as above:</p>
<pre><code>Initial arrangement:
811589153, 1623178306, -2434767459, 2434767459, -1623178306, 0, 3246356612

After 1 round of mixing:
0, -2434767459, 3246356612, -1623178306, 2434767459, 1623178306, 811589153

After 2 rounds of mixing:
0, 2434767459, 1623178306, 3246356612, -2434767459, -1623178306, 811589153

After 3 rounds of mixing:
0, 811589153, 2434767459, 3246356612, 1623178306, -1623178306, -2434767459

After 4 rounds of mixing:
0, 1623178306, -2434767459, 811589153, 2434767459, 3246356612, -1623178306

After 5 rounds of mixing:
0, 811589153, -1623178306, 1623178306, -2434767459, 3246356612, 2434767459

After 6 rounds of mixing:
0, 811589153, -1623178306, 3246356612, -2434767459, 1623178306, 2434767459

After 7 rounds of mixing:
0, -2434767459, 2434767459, 1623178306, -1623178306, 811589153, 3246356612

After 8 rounds of mixing:
0, 1623178306, 3246356612, 811589153, -2434767459, 2434767459, -1623178306

After 9 rounds of mixing:
0, 811589153, 1623178306, -2434767459, 3246356612, 2434767459, -1623178306

After 10 rounds of mixing:
0, -2434767459, 1623178306, 3246356612, -1623178306, 2434767459, 811589153
</code></pre>

<p>The grove coordinates can still be found in the same way. Here, the 1000th number after <code>0</code> is <code><em>811589153</em></code>, the 2000th is <code><em>2434767459</em></code>, and the 3000th is <code><em>-1623178306</em></code>; adding these together produces <code><em>1623178306</em></code>.</p>
<p>Apply the decryption key and mix your encrypted file ten times. <em>What is the sum of the three numbers that form the grove coordinates?</em></p>
</article>


In [48]:
tests = [
    {
        "name": "Example 1",
        "s": """
            1
            2
            -3
            3
            -2
            0
            4
        """,
        "expected": 1623178306,
    },
]


class DecryptorII(Decryptor):
    def __init__(self, s) -> None:
        super().__init__(s)

        for _ in range(9):
            self._mix()

    @classmethod
    def _create_list(cls, s) -> list[Node]:
        nodes = super()._create_list(s)

        key = 811589153
        for n in nodes:
            n.value *= key

        return nodes


@test(tests=tests)
def test_part_II(s: str) -> int:
    return DecryptorII(s).grove_sum()


Test Example 1 passed, for test_part_II.
Success


In [49]:
print(f"PartI I: {DecryptorII(puzzle).grove_sum()}")

PartI I: 656575624777


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>656575624777</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
